In [1]:
import sys
import numpy as np
import cv2
from pathlib import Path
import Test2

from PyQt5.QtCore import QRectF, Qt, QPointF, QT_VERSION_STR, QSize
from PyQt5.QtGui import QPen, QBrush, QColor, QImage, QPainter, QPixmap
from PyQt5.QtPrintSupport import QPrinter
from PyQt5.QtWidgets import QMainWindow, QGridLayout, QDialog, QGraphicsView, QApplication, QPushButton, QGraphicsScene, \
    QFileDialog, QWidget, QLineEdit, QLabel
from PyQt5.uic.Compiler.qtproxies import QtGui

class Paint(QGraphicsView):
    def __init__(self):
        QGraphicsView.__init__(self)
        self.setSceneRect(QRectF(self.viewport().rect()))
        self.scene = QGraphicsScene()

        self.isPaint = False
        self.isDelete = False
        self.isClear = False

    # def drawBackground(self, painter: QPainter, rect: QRectF):
    #     bursh = QBrush(Qt.black)
    #     painter.fillRect(rect, bursh)


    def tools(self,e):
        if self.isPaint == True:
            pen = QPen(Qt.red)
#             brush = QBrush(Qt.black)
            pen.setWidth(10)
            
            self.scene.addLine(self.xPre,self.yPre,e.x(),e.y(),pen)
            self.xPre = e.x()
            self.yPre = e.y();
            
            self.setScene(self.scene)
        if self.isDelete == True:
            items = self.items(e.x(),e.y())
            for item in items:
                self.scene.removeItem(item)
        
#         if self.isClear == True:
#             self.scene.clear()
#             QGraphicsView.__init__(self)
#             self.setSceneRect(QRectF(self.viewport().rect()))
#             self.scene = QGraphicsScene()
                
            
        
    def mousePressEvent(self, event: QtGui.QMouseEvent):
#         print("mouse press: ",event.pos().x())
        e = QPointF(self.mapToScene(event.pos()))
        self.xPre = e.x()
        self.yPre = e.y();
        self.tools(e)

    def mouseMoveEvent(self, event: QtGui.QMouseEvent):
#         print("mouse move: ",event.pos())
        e = QPointF(self.mapToScene(event.pos()))
        self.tools(e)

    # def saveImage(self):
    #     fname,filter = QFileDialog.getSaveFileName(self, 'Save File', r"D:\python_pyCharm\testPyqt", "Images (*.png)")
    #     if fname:
    #         print(f"file name: {fname}")
    #         self.scene.clearSelection()
    #         self.setSceneRect(self.scene.itemsBoundingRect())
    #         print("size rect: ", self.scene.sceneRect().size().toSize())
    #         # self.imgsaved = QImage(self.scene.sceneRect().size().toSize(), QImage.Format_ARGB32)
    #         self.imgsaved = QImage(QSize(64,64), QImage.Format_ARGB32)
    #         self.imgsaved.fill(Qt.transparent)
    #         painter = QPainter(self.imgsaved)
    #         painter.setRenderHint(QPainter.HighQualityAntialiasing)
    #         self.scene.render(painter)
    #         self.imgsaved.save(fname)
    #
    #     else:
    #         print("Duong dan sai")
    
    def predict(self):
        name_image = ".\image\predict.png"
        self.scene.clearSelection()
        self.setSceneRect(self.scene.itemsBoundingRect())
        print("size rect: ", self.scene.setSceneRect.size().toSize())
        self.imgsaved = QImage(self.scene.sceneRect().size().toSize(), QImage.Format_ARGB32)
    
#         self.imgsaved = QImage(QSize(64, 64), QImage.Format_ARGB32)
        self.imgsaved.fill(Qt.transparent)
        

        painter = QPainter(self.imgsaved)
        painter.setRenderHint(QPainter.HighQualityAntialiasing)
        self.scene.render(painter)
        self.imgsaved.save(name_image)


        p = Path.cwd()
        p = p.joinpath(".\image\predict.png")
    
        if p.is_file():
            test = Test2.Test(p)
            word_and_acc = test.predict()
            return word_and_acc
        else:
            raise ValueError("file không tồn tại")






class MyDialog(QDialog):
    def __init__(self):
        QDialog.__init__(self)
        self.resize(800,900)
        self.layout=QGridLayout()
        self.setLayout(self.layout)
        self.paint = Paint()

        # self.result_text = QLineEdit()
        # # self.result_text.
        # self.result_text.setPlaceholderText("kết quả")
        # self.result_text.setDisabled(1)

        self.result = QLabel()
        self.result.setText("Ket qua")
        self.result.setStyleSheet("font:18pt;")
        self.result.setAlignment(Qt.AlignCenter)

        self.btn_paint =QPushButton("Paint")
        self.btn_delete = QPushButton("Del")
        self.btn_clear = QPushButton("Clear")
        self.btn_save = QPushButton("Predict")

        self.layout.addWidget(self.btn_paint,0,0,1,2)
        self.layout.addWidget(self.btn_delete,1,0,1,2)
        self.layout.addWidget(self.btn_clear,2,0,1,2)
        # self.layout.addWidget(self.result_text,3,0)
        self.layout.addWidget(self.btn_save,3,1)
        self.layout.addWidget(self.result,3,0)
        self.layout.addWidget(self.paint,5,0,1,2)

        self.btnDefault = "background-color: gray; border:0;padding:20px"
        self.btnActive = "background-color: orange; border:0;padding:20px"

        self.btn_paint.setStyleSheet(self.btnDefault)
        self.btn_delete.setStyleSheet(self.btnDefault)
        self.btn_clear.setStyleSheet(self.btnDefault)
        self.btn_save.setStyleSheet(self.btnDefault)

        self.btn_paint.clicked.connect(self.isPaint)
        self.btn_delete.clicked.connect(self.isDelete)
        self.btn_clear.clicked.connect(self.isClear)
        self.btn_save.clicked.connect(self.isSave)
    #
    def resizeEvent(self, a0: QtGui.QResizeEvent):
        self.paint.setSceneRect(QRectF(self.paint.viewport().rect()))
    def isPaint(self):
        if self.paint.isPaint == False:
            self.paint.isPaint=True
            self.btn_paint.setStyleSheet(self.btnActive)
        else:
            self.paint.isPaint=False
            self.btn_paint.setStyleSheet(self.btnDefault)

        self.paint.isDelete =False
        self.paint.isClear= False
        self.btn_clear.setStyleSheet(self.btnDefault)
        self.btn_delete.setStyleSheet(self.btnDefault)
    def isDelete(self):
        if self.paint.isDelete == False:
            self.paint.isDelete=True
            self.btn_delete.setStyleSheet(self.btnActive)
        else:
            self.paint.isDelete=False
            self.btn_delete.setStyleSheet(self.btnDefault)

        self.paint.isClear =False
        self.paint.isPaint= False
        self.btn_paint.setStyleSheet(self.btnDefault)
        self.btn_clear.setStyleSheet(self.btnDefault)
    def isClear(self):
        if self.paint.isClear == False:
            self.paint.isClear=True
            self.btn_clear.setStyleSheet(self.btnActive)
        else:
            self.paint.isClear=False
            self.btn_clear.setStyleSheet(self.btnDefault)

        self.paint.isDelete =False
        self.paint.isPaint=False
        self.btn_delete.setStyleSheet(self.btnDefault)
        self.btn_paint.setStyleSheet(self.btnDefault)
        self.paint.scene.clear()
        self.paint.viewport().update()
        self.paint.setSceneRect(QRectF(self.paint.viewport().rect()))
        self.scene = QGraphicsScene()
        
#         self.btn_clear.setStyleSheet(self.btnDefault)
        self.btn_paint.click()

    def isSave(self):
        word_and_acc = self.paint.predict()
#         print("isSave")
        print(word_and_acc)
        word =word_and_acc[0]
        acc = word_and_acc[1]
        # if acc>=0.8:
        text = word+" "+str(round(acc,4))
        self.result.setText(text)

app=QApplication(sys.argv)
dialog = MyDialog();
dialog.show()
app.exec_()

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


size rect:  PyQt5.QtCore.QSize(199, 268)
path_model_last: ..\a_model_khoa_luan\model\modelTest.ckpt-2
INFO:tensorflow:Restoring parameters from ..\a_model_khoa_luan\model\modelTest.ckpt-2
(?, 64, 64, 1)


<Figure size 640x480 with 1 Axes>

['シ ', 0.94751704]
size rect:  PyQt5.QtCore.QSize(293, 46)
path_model_last: ..\a_model_khoa_luan\model\modelTest.ckpt-2
INFO:tensorflow:Restoring parameters from ..\a_model_khoa_luan\model\modelTest.ckpt-2
(?, 64, 64, 1)


<Figure size 640x480 with 1 Axes>

['ユ ', 0.912354]
size rect:  PyQt5.QtCore.QSize(13, 123)
path_model_last: ..\a_model_khoa_luan\model\modelTest.ckpt-2
INFO:tensorflow:Restoring parameters from ..\a_model_khoa_luan\model\modelTest.ckpt-2
(?, 64, 64, 1)


<Figure size 640x480 with 1 Axes>

[' ト', 0.5970014]


0